<a href="https://colab.research.google.com/github/Xornotor/PPGEEC-CompEvolutiva-Atividades/blob/main/Avalia%C3%A7%C3%A3o_1A_Algoritmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# **Avaliação 1A - Computação Evolutiva e Meta-heurísticas**

**Docente:** Prof. Dr. Edmar Egídio Purcino de Souza

**Discentes:** André Paiva, Gabriel Lucas, Márcio Barros e Shaísta Câmara

## 1 - Inicialização e definição de funções do algoritmo genético

### 1.1 - Importação de Dependências e Parametrização

In [ ]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from plotly import graph_objects as go
from plotly import express as px
from itertools import combinations
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['figure.dpi'] = 150

In [ ]:
general_conditions = {
    # Parâmetros do algoritmo genético

    #------------------------------------------------------------------
    # Escolha a função de fitness a ser usada: "dropwave" ou "levi"
    "functionName": "dropwave",
    #"functionName": "levi",
    #------------------------------------------------------------------

    # Quantidade de indivíduos por geração
    "populationSize": 100,

    # Quantidade máxima de gerações
    "generationCount": 100,

    # Quantidade de indivíduos a ser preservados no elitismo
    "elitism": 10,

    # Seleção: 1 = Roleta, 2 = Torneio, 3 = Aleatório (50/50)
    "selectionMethod": 3,

    # Recombinação: 1 = Aritmética, 2 = Discreta, 3 = Aleatório (50/50)
    "recombMethod": 3,

    # Taxa de recombinação
    "recombRate": 0.8,

    # Taxa de mutação
    "mutRate": 0.5,

    # Aumento da taxa de mutação em caso de baixa diversidade
    "mutEnhance": True,
    "mutEnhanceSetpoint": 5,
    "mutEnhanceRate": 0.8,

    # Reinicialização da população em caso de baixa diversidade
    "popReset": True,
    "popResetSetpoint": 1,

    # Parada antecipada em caso de convergência
    "earlyStop": True,
    "earlyStopPatience": 10,

    # Quantidade de genes por cromossomo
    "chromosomeLength": 2,

    # Limites de valores para os genes
    "lowerBound": -100,
    "upperBound": 100
}

### 1.2 - Definição de função objetivo (*fitness*)

In [ ]:
def fitness(chromosome, conditions):
    # Dropwave
    x1 = chromosome[0]
    x2 = chromosome[1]
    if conditions["functionName"] == "dropwave":
        r = np.sqrt(x1**2 + x2**2)
        numerator = -(1 + np.cos(12 * r))
        denominator = 0.5 * (x1**2 + x2**2) + 2
        return numerator / denominator
    elif conditions["functionName"] == "levi":
        term1 = np.sin(3 * np.pi * x1)**2
        term2 = (x1 - 1)**2 * (1 + np.sin(3 * np.pi * x2)**2)
        term3 = (x2 - 1)**2 * (1 + np.sin(2 * np.pi * x2)**2)
        return term1 + term2 + term3
    else:
        raise ValueError("Função de fitness desconhecida.")

In [ ]:
# Plot interativo da função objetivo
def fitness_plot(conditions):
    if conditions["functionName"] == "dropwave":
        x = y = np.linspace(-5, 5, 500)
    if conditions["functionName"] == "levi":
        x = y = np.linspace(-10, 10, 500)

    chromosome = np.meshgrid(x, y)
    Z = fitness(chromosome, conditions)

    fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y, colorscale='rainbow')])
    fig.update_layout(title=f"Função {conditions['functionName'].capitalize()}",
                    autosize=False, width=800, height=600,
                    margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

### 1.3 - Funções auxiliares para o algoritmo genético

In [ ]:
# Instanciação de população inicial
def generate_population(conditions):
    population = [(np.random.uniform(conditions["lowerBound"],
                                     conditions["upperBound"],
                                     conditions["chromosomeLength"],), 0)
                                     for _ in range(conditions["populationSize"])]
    population = [(np.array(ind), fitness(ind, conditions)) for ind, _ in population]
    return population

# Medidas de diversidade
def diversity_metrics(population):
    genes = np.array([chromosome for chromosome, _ in population])
    std_devs = np.std(genes, axis=0)
    mean_distance_to_center = np.mean(np.linalg.norm(genes - np.mean(genes, axis=0), axis=1))
    distances = [np.linalg.norm(a - b) for a, b in combinations(genes, 2)]
    mean_pairwise_distance = np.mean(distances)
    return std_devs, mean_distance_to_center, mean_pairwise_distance

# Seleção
def selection(population, conditions):
    selected_parents = []
    random_sel = np.random.uniform(0, 1) # Seleção aleatória: > 0.5 Torneio, c.c. Roleta

    if(conditions["selectionMethod"] == 1 or (conditions["selectionMethod"] == 3 and random_sel <= 0.5)):
        pop_fit = np.array([indiv[1] for indiv in population])
        pop_argsort = np.argsort(pop_fit)
        pop_fit = pop_fit[pop_argsort]
        pop_fit = pop_fit[::-1]
        pop_reorg = [population[i] for i in pop_argsort]
        fit_rescale = np.logspace(0, 1, len(pop_fit))[::-1]
        fit_rescale /= np.sum(fit_rescale)
        fit_rescale = np.add.accumulate(fit_rescale)
        fit_rescale[-1] = 1.0
        fit_rescale = np.sqrt(fit_rescale)
        pop_rescale = [(pop_reorg[i][0], fit_rescale[i]) for i in range(len(pop_reorg))]
        for _ in range(len(pop_rescale)):
            rand_parent = np.random.uniform(0, 1)
            for i in range(len(pop_rescale)):
                if rand_parent <= pop_rescale[i][1]:
                    selected_parents.append(pop_rescale[i][0])
                    break
    elif(conditions["selectionMethod"] == 2 or (conditions["selectionMethod"] == 3 and random_sel > 0.5)):
        tournament_size = 10
        for _ in range(len(population)):
            tournament = random.sample(population, tournament_size-1)
            winner = min(tournament, key=lambda x: x[1]) # x[1] é o fitness na population/tournament
            selected_parents.append(winner[0]) # winner[0] pega somente os valores dos genes (x1,x2)
    else:
        raise ValueError("selectionMethod inválido.")

    best_individual = min(population, key=lambda x: x[1])[0]
    selected_parents.append(best_individual)
    return selected_parents

# Recombinação
def crossover(parent1, parent2, conditions):
    if np.random.uniform(0, 1) < conditions["recombRate"]:
        random_cross = np.random.uniform(0, 1) # Seleção aleatória: > 0.5 Discreta, c.c. Aritmética
        if(conditions["recombMethod"] == 1 or (conditions["recombMethod"] == 3 and random_cross <= 0.5)):
            interp = np.random.uniform(0, 1)
            child1 = (interp * parent1) + ((1 - interp) * parent2)
            child2 = (interp * parent2) + ((1 - interp) * parent1)
            return child1, child2
        elif(conditions["recombMethod"] == 2 or (conditions["recombMethod"] == 3 and random_cross > 0.5)):
            crossover_point = random.randint(1, conditions["chromosomeLength"] - 1)
            child1 = np.append(parent1[:crossover_point], parent2[crossover_point:])
            child2 = np.append(parent2[:crossover_point], parent1[crossover_point:])
            return child1, child2
        else:
            raise ValueError("recombMethod inválido.")
    else:
        return parent1, parent2

# Mutação
def mutate(individual, low_diversity, conditions):
    mutated_individual = individual.copy()
    mut_rate = conditions["mutEnhanceRate"] if low_diversity else conditions["mutRate"]
    for i in range(conditions["chromosomeLength"]):
        if np.random.uniform(0, 1) < mut_rate:
            mutated_individual[i] += np.random.uniform(-1, 1)
    return np.clip(mutated_individual, conditions["lowerBound"], conditions["upperBound"])

In [ ]:
# FUNÇÕES DE PLOT

# Plot das curvas de convergência

def convergence_curve_individual(population_df, conditions):
    x = np.arange(0, len(population_df))
    y = population_df["Fitness"].to_numpy()

    fig = plt.figure()
    ax = plt.gca()

    plt.plot(x, y, color='Plum', linestyle='--', linewidth=2,
            marker='o', markeredgewidth=1, markerfacecolor='FireBrick',
            markeredgecolor='black', markersize=2)

    plt.xlabel('Geração')
    plt.ylabel('Função objetivo')
    plt.xticks(np.linspace(0, len(population_df), 8, dtype=int))

    # Mostra no título qual função está sendo usada
    plt.title(f'Curva de convergência - Função {conditions["functionName"].capitalize()}')

    if(conditions["functionName"] == "dropwave"):
        plt.axhline(y = -1, color ="red", linestyle ="-", zorder=1.5, linewidth=2)
    elif(conditions["functionName"] == "levi"):
        plt.axhline(y = 0, color ="red", linestyle ="-", zorder=1.5, linewidth=2)

    plt.grid(True)
    plt.show()
    plt.close(fig)

    # Identificar última mudança significativa (> 1e-4) no fitness
    threshold = 1e-4
    last_significant_idx = 0

    for i in range(1, len(y)):
        if abs(y[i] - y[i - 1]) > threshold:
            last_significant_idx = i

    print(f"Última mudança significativa (>1e-4) ocorreu na Geração de Convergência: {last_significant_idx}")
    print(f"Best Fitness : {y[-1]:.6f}")
    print(f"POPULATION_SIZE: {conditions['populationSize']}")
    print(f"GENERATION_COUNT: {conditions['generationCount']}")
    print(f"RECOMB_RATE: {conditions['recombRate']}")
    print(f"MUTATION_RATE: {conditions['mutRate']}")
    print(f"CHROMOSOME_LENGTH: {conditions['chromosomeLength']}")
    print(f"LOWER_BOUND: {conditions['lowerBound']}")
    print(f"UPPER_BOUND: {conditions['upperBound']}")

# Plot das medidas de diversidade
def plot_diversity_metrics(std_devs_list, mean_distance_to_center_list, mean_pairwise_distance_list):
    generations = range(len(std_devs_list))
    std_x1 = [std[0] for std in std_devs_list]
    std_x2 = [std[1] for std in std_devs_list]

    plt.figure(figsize=(10, 3))

    # Gráfico 1: Desvio padrão dos genes
    plt.subplot(1, 3, 1)
    plt.plot(generations, std_x1, label='Std x1')
    plt.plot(generations, std_x2, label='Std x2')
    plt.title('Desvio Padrão dos Genes')
    plt.xlabel('Geração')
    plt.ylabel('Desvio Padrão')
    plt.legend()
    plt.grid(True)

    # Gráfico 2: Distância ao centro
    plt.subplot(1, 3, 2)
    plt.plot(generations, mean_distance_to_center_list, color='orange')
    plt.title('Distância Média ao Centro')
    plt.xlabel('Geração')
    plt.ylabel('Distância')
    plt.grid(True)

    # Gráfico 3: Distância média entre pares
    plt.subplot(1, 3, 3)
    plt.plot(generations, mean_pairwise_distance_list, color='green')
    plt.title('Distância Média entre Indivíduos')
    plt.xlabel('Geração')
    plt.ylabel('Distância')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

### 1.4 - Algoritmo Genético

In [ ]:
def genetic_algorithm(conditions, iter_print=False):
    # Inicialização da primeira geração
    population = generate_population(conditions)

    # Lista de melhores valores de cada geração
    best_dicts_list = []

    # Histórico da sociedade
    population_df = pd.DataFrame()

    # Variáveis de controle para early stopping
    last_iter_with_enhancement = 0
    fitness_for_last_iter_with_enhance = np.inf

    # Medidas de diversidade
    diversity_dicts_list = []
    std_devs_list = []
    mean_distance_to_center_list = []
    mean_pairwise_distance_list = []
    std_devs = dist_to_center = pairwise_dist = np.inf

    # Taxa de mutação (pode aumentar em caso de baixa diversidade)
    low_diversity = False

    for iter in range(conditions["generationCount"]):
        # Avalia a população com a função de fitness escolhida
        population = [(chromosome, fitness(chromosome, conditions)) for chromosome, _ in population]

        gen_dicts = []
        for i, item in enumerate(population):
            pop_dict = {"Generation": iter, "Fitness": item[1]}
            for j, gene in enumerate(item[0]):
                pop_dict[f"x{j+1}"] = gene
            gen_dicts.append(pop_dict)
        population_df = pd.concat([population_df, pd.DataFrame(gen_dicts)], ignore_index=True)

        # Seleciona os pais
        parents = selection(population, conditions) # retorna um vetor de size population+1, isso é importante pro loop seguinte funcionar, como foi feito

        # Realiza o crossover e a mutação para gerar descendentes
        offspring = []
        # Vai de 2 em 2 dentro da população
        for i in range(0, conditions["populationSize"], 2):
            parent1 = np.array(parents[i])
            parent2 = np.array(parents[i + 1])
            child1, child2 = crossover(parent1, parent2, conditions)
            child1 = mutate(child1, low_diversity, conditions)
            child2 = mutate(child2, low_diversity, conditions)
            offspring.extend([child1, child2])

        # Recria a população a partir dos descendentes (sem avaliar fitness)
        population_x = [(chromosome, 0) for chromosome in offspring]

        # Elitismo: mantém os melhores indivíduos da geração anterior
        elites = sorted(population, key=lambda x: x[1])[:conditions["elitism"]]

        best_individual = min(population, key=lambda x: x[1])[0] # O [0] no final é para pegar apenas o cromossomo, depois de avaliar o menor fitness da população
        population = elites + population_x
        population = population[:conditions["populationSize"]]

        # Reavalia o melhor indivíduo para registrar
        fitness_value = fitness(best_individual, conditions)
        best_dict = {"Generation": iter, "Fitness": fitness(best_individual, conditions)}
        for i, gene in enumerate(best_individual):
            best_dict[f"x{i+1}"] = gene
        best_dicts_list.append(best_dict)

        # Calcula diversidade da população atual
        std_devs, dist_to_center, pairwise_dist = diversity_metrics(population)
        diversity_dicts_list.append({"Generation": iter,
                                     "std_dev": std_devs,
                                     "dist_to_center": dist_to_center,
                                     "pairwise_dist": pairwise_dist})

        # Aumento da taxa de mutação em caso de baixa diversidade
        low_diversity = conditions["mutEnhance"] and pairwise_dist < conditions["mutEnhanceSetpoint"]

        # Reinicialização da população em caso de baixa diversidade
        if(conditions["popReset"] and pairwise_dist < conditions["popResetSetpoint"]):
            population = elites + generate_population(conditions)
            population = population[:conditions["populationSize"]]
            std_devs, dist_to_center, pairwise_dist = diversity_metrics(population)

        # Armazena os valores
        std_devs_list.append(std_devs)
        mean_distance_to_center_list.append(dist_to_center)
        mean_pairwise_distance_list.append(pairwise_dist)

        if(iter_print):
            print(f"Generation: {iter} | Best Fitness: {fitness_value:.6f} | Best Solution: {best_individual}")

        # Early stopping
        if(fitness_value < fitness_for_last_iter_with_enhance):
            last_iter_with_enhancement = iter
            fitness_for_last_iter_with_enhance = fitness_value
        elif(conditions["earlyStop"] and iter - last_iter_with_enhancement >= conditions["earlyStopPatience"]):
            break

    best_result = best_dicts_list[-1]
    best_df = pd.DataFrame(best_dicts_list).sort_values("Generation")
    diversity_df = pd.DataFrame(diversity_dicts_list).sort_values("Generation")

    return best_result, best_df, diversity_df, population_df

## 2 - Testes do Algoritmo Genético: Execução única

In [ ]:
fitness_plot(general_conditions)

In [ ]:
best_result, best_df, diversity_df, population_df = genetic_algorithm(general_conditions, iter_print=True)

In [ ]:
px.scatter(population_df, "x1", "x2", "Fitness", animation_frame="Generation",
           width=800, height=600, color_continuous_scale='bluered',
           title=f"Função {general_conditions['functionName'].capitalize()} - Evolução das Gerações")

In [ ]:
plot_diversity_metrics(diversity_df["std_dev"].to_numpy(),
                       diversity_df["dist_to_center"].to_numpy(),
                       diversity_df["pairwise_dist"].to_numpy())

In [ ]:
convergence_curve_individual(best_df, general_conditions)

## 3 - Definição de funções para os experimentos estatísticos

In [ ]:
# Conversão para camelCase para nomes de excursion vars
def to_camel_case(text):
    s = text.replace("-", " ").replace("_", " ")
    s = s.split()
    if len(text) == 0:
        return text
    return s[0].lower() + ''.join(i.capitalize() for i in s[1:])

# Teste com diversas inicializações em diferentes condições
def statistical_test(conditions, excursion_var, excursion_array, initializations=20):
    best_result_dicts = []
    final_pop_df = pd.DataFrame()
    internal_conditions = conditions.copy()
    for i in excursion_array:
        internal_conditions[to_camel_case(excursion_var)] = i
        for j in range(initializations):
            print(f"\r{excursion_var} {i}, Initialization {j}...    ", end='')
            result, pop_df, _, _ = genetic_algorithm(internal_conditions)
            result[excursion_var] = i
            result["Initialization"] = j
            pop_df[excursion_var] = i
            pop_df["Initialization"] = j
            best_result_dicts.append(result)
            final_pop_df = pd.concat([final_pop_df, pop_df], ignore_index=True)
    final_result_df = pd.DataFrame(best_result_dicts)
    print("\rDone.")
    return final_result_df, final_pop_df

# Violinplot de fitness por variável de excursionamento
def violin_fitness(conditions, excursion_var, result_df):
    fig, ax = plt.subplots(1, 1)
    sns.violinplot(data=result_df, x=excursion_var, y="Fitness",
                split=True, ax=ax, inner="quart", density_norm="count")
    ax.set_title(f"Violinplot Normalizado - Função {conditions['functionName'].capitalize()}")
    fig.show()

# Plot de curva de convergência para a inicialização com melhor fitness de cada condição
def convergence_curve_statistical(conditions, excursion_var, result_df, pop_df, figsize=(8, 8)):
    min_fitness_indices = result_df.groupby(excursion_var)['Fitness'].idxmin()
    best_init_per_var = result_df.loc[min_fitness_indices][[excursion_var, "Initialization"]]

    plot_amount = len(best_init_per_var)

    if(len == 3):
        h = 3
        v = 1
    else:
        h = int(np.ceil(np.sqrt(plot_amount)))
        v = int(np.ceil(plot_amount / h))

    fig, ax = plt.subplots(v, h, figsize=figsize)
    fig.suptitle(f'Curva de convergência - Função {conditions["functionName"].capitalize()} (Melhor inicialização por {excursion_var})')
    for i, var_value in enumerate(best_init_per_var[excursion_var]):
        best_init = best_init_per_var[best_init_per_var[excursion_var] == var_value]["Initialization"].to_numpy()[0]
        pop_subdf = pop_df[(pop_df[excursion_var] == var_value) & (pop_df["Initialization"] == best_init)]
        x = np.arange(0, len(pop_subdf))
        y = pop_subdf["Fitness"].to_numpy()
        ax[i//h, i%h].plot(x, y, color='Plum', linestyle='--', linewidth=2,
            marker='o', markeredgewidth=1, markerfacecolor='FireBrick',
            markeredgecolor='black', markersize=2)
        ax[i//h, i%h].set_xlabel('Geração')
        ax[i//h, i%h].set_ylabel('Função objetivo')
        ax[i//h, i%h].set_xticks(np.linspace(0, len(pop_subdf), 8, dtype=int))
        ax[i//h, i%h].set_title(f'{excursion_var}: {var_value}')
        if(conditions["functionName"] == "dropwave"):
            ax[i//h, i%h].axhline(y = -1, color ="red", linestyle ="-", zorder=1.5, linewidth=2)
        elif(conditions["functionName"] == "levi"):
            ax[i//h, i%h].axhline(y = 0, color ="red", linestyle ="-", zorder=1.5, linewidth=2)
        ax[i//h, i%h].grid(True)
    fig.set_tight_layout("tight")
    plt.show()
    plt.close(fig)

## 4 - Experimentos estatísticos

### 4.1 - Experimento 1: Efeitos do tamanho da população (Dropwave)

In [ ]:
exp1_conditions = {
    "functionName": "dropwave",
    "populationSize": 10,
    "generationCount": 50,
    "elitism": 2,
    "selectionMethod": 2, # Torneio
    "recombMethod": 1, # Aritmética
    "recombRate": 0.8,
    "mutRate": 0.5,
    "mutEnhance": False,
    "mutEnhanceSetpoint": 5,
    "mutEnhanceRate": 0.8,
    "popReset": False,
    "popResetSetpoint": 1,
    "earlyStop": False,
    "earlyStopPatience": 10,
    "chromosomeLength": 2,
    "lowerBound": -100,
    "upperBound": 100
}

In [ ]:
exp1_result_df, exp1_pop_df = statistical_test(exp1_conditions, "Population Size", np.arange(20, 121, 20))

In [ ]:
exp1_result_df.groupby(["Population Size"])[["x1", "x2", "Fitness"]].agg(["mean", "std"])

In [ ]:
violin_fitness(exp1_conditions, "Population Size", exp1_result_df)

In [ ]:
convergence_curve_statistical(exp1_conditions, "Population Size", exp1_result_df, exp1_pop_df, figsize=(10, 6))

### 4.2 - Experimento 2: Efeitos do tamanho da população (Levi)

In [ ]:
exp2_conditions = {
    "functionName": "levi",
    "populationSize": 20,
    "generationCount": 50,
    "elitism": 2,
    "selectionMethod": 2, # Torneio
    "recombMethod": 1, # Aritmética
    "recombRate": 0.8,
    "mutRate": 0.5,
    "mutEnhance": False,
    "mutEnhanceSetpoint": 5,
    "mutEnhanceRate": 0.8,
    "popReset": False,
    "popResetSetpoint": 1,
    "earlyStop": False,
    "earlyStopPatience": 10,
    "chromosomeLength": 2,
    "lowerBound": -100,
    "upperBound": 100
}

In [ ]:
exp2_result_df, exp2_pop_df = statistical_test(exp2_conditions, "Population Size", np.arange(20, 121, 20))

In [ ]:
exp2_result_df.groupby(["Population Size"])[["x1", "x2", "Fitness"]].agg(["mean", "std"])

In [ ]:
convergence_curve_statistical(exp2_conditions, "Population Size", exp2_result_df, exp2_pop_df, figsize=(10, 6))

In [ ]:
violin_fitness(exp2_conditions, "Population Size", exp2_result_df)